In [10]:
import numpy as np

# We first import the time module to compute runtime
import time
start_time = time.time()


# First, we write the function for creating box
# This is the code to make a box around a vector v = [v1 v2 ... vn]^T with parameters r = [r1 r2 ... rn]^T
def make_box(v, r):
    n = len(v)
    zeros = [0]*(n+1)
    mat = []
    # Adding the constraints for positive values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)
    # Adding the constraints for negative values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)   
    # print('mat zeros = ', mat) # [[0.1,0,0], [0.1,0,0], [0.1,0,0], [0.1,0,0]]
    
    for i in range(n):
        mat[i][i+1] = 1 # [[0.1,1,0], [0.1,0,1], [0.1,0,0], [0.1,0,0]]
    # print('mat pos = ', mat)
    j = 1
    for i in range(n,2*n):
        mat[i][j] = -1 # # [[0.1,1,0], [0.1,0,1], [0.1,-1,0], [0.1,0,-1]]
        j += 1
    # print('mat for constraints = ', mat)
    Box = Polyhedron(ieqs=mat, backend='ppl', base_ring=QQ)
    return Box


# Select the file to take input for algorithm 1
v = input("Enter the example number to run algorithm 1: ")
exp_num = v
# Open the input file
if (int(v)==1):
    input_file = open(r"inputs/ex1_dummy.txt")
elif (int(v)==2):
    input_file = open(r"inputs/ex2_balance_system.txt")
elif (int(v)==3):
    input_file = open(r"inputs/ex3_3deg_quadcopter.txt")
elif (int(v)==4):
    input_file = open(r"inputs/ex4_6deg_quadcopter.txt")

# Execute the input file to get A, B, K, X0, U0
exec(input_file.read())
input_file.close()

# Create the box
v = [0]*(2*n)
r = [ep]*n
zeros = [0]*n
rbox = zeros+r
Box = make_box(v,rbox)
# print('Box = ', Box.vertices())

# Converting K into K = [[I 0] [0 K]]
xn, un = n, u
K1 = [[0 for i in range(xn)]for j in range(xn)]
KI = np.identity(xn)
K1 = np.hstack((KI, K1))
K2 = [[0 for i in range(xn)] for j in range(un)]
K2 = np.hstack((K2,K))
K = Matrix(np.vstack((K1,K2)))

# At first, we define C = [A B]
C = Matrix(np.hstack((A,B)))

# Next, we have input P0 = [X0 U0]^T
P0 = [np.hstack((x,u)) for x in X0.vertices() for u in U0.vertices()]
P0 = Polyhedron(vertices=P0)

# We define the matrix M as follows,
I = np.identity(xn)
M = Matrix(np.vstack((I,I)))

# Opening a file to write the outputs
exp_string = "outputs/alg2/ex" + exp_num + ".txt"
f = open(exp_string, "a")
f.write('\nNew Experiment:\n')

# Running algorithm 2
Pc = P0
iter = 1
for i in range(iter):
    try:
        print('i = ', i, file=f)
        Xc = C*Pc
        print('Number_of_Cons(Xc) = ', len(list(Xc.Hrepresentation())), file=f)
        Xc_twice = M*Xc
        print('Number_of_Cons(Xc_twice) = ', len(list(Xc_twice.Hrepresentation())), file=f)
        Xh = Xc_twice + Box
        print('Number_of_Cons(Xh) = ', len(list(Xh.Hrepresentation())), file=f)
        Pc = K*Xh
        print('Number_of_Cons(Pc) = ', len(list(Pc.Hrepresentation())), file=f)
        print('Vol(Xc) = ', float(Xc.volume()),  file=f)
        print('Vol(Xh) = ', float(Xh.volume()),  file=f)
        print('Vol(Pc) = ', float(Pc.volume()),  file=f)
    except ValueError as ve:
        print('ValueError = ', ve, file=f)
        break
    except ZeroDivisionError as ze:
        print("ZeroDivisionError = ", ze, file=f)

# Print the output State
print("\nThe constraints of output Xc = ", list(Xc.Hrepresentation()), file=f)

# Save the runtime
end_time = time.time()
total_time_in_ms = (end_time-start_time)*10**3
f.write('\nTotal runtime = '+str(round(total_time_in_ms, 4))+' ms for '+str(iter)+' iterations\n')

# Make the string and output
out_titles = "Iteration, Num_Cons(Xout), Num_Ver(Xout), Dimensions(Xout), Runtime (ms)\n"
f.write(out_titles)
out_values = str([iter, len(list(Xc.Hrepresentation())), len(Xc.vertices_list()), len(Xc.vertices_list()[0]), total_time_in_ms])[1:-1]
f.write(out_values)

# Close the file
f.close()

# Saving the plot
try:
    fig = Xc.plot()
    fig_string = "outputs/alg2/ex" + exp_num + ".png"
    fig.save(fig_string)
except:
    pass

# Plotting in notebook
Xc


KeyboardInterrupt: 